In [128]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import polars as pl
import pandas as pd

birdy_uri = 'spotify:artist:1l2ekx5skC4gJH8djERwh1'
credentials = SpotifyClientCredentials(client_id="e12321d0c25843b78c739eb124ac8d2e", client_secret="6c1e01f319f84545afaeacb6ef7452fd")
spotify = spotipy.Spotify(client_credentials_manager=credentials, auth=credentials.get_access_token(as_dict=False))


In [131]:
playlist_tracks = spotify.playlist_tracks("spotify:playlist:37i9dQZEVXbdCnni12q6yv")

# configure DataFrame full width
pl.Config.set_fmt_str_lengths(100)

# create DataFrame based on JSON Schema
results = pl.DataFrame(playlist_tracks['items'])

# extract relevant fields from nested JSON Schema
results = results.with_columns(pl.col('track').struct.field('uri').alias('track_uri')).with_columns(pl.col('track').struct.field('name').alias('track_name')).with_columns(pl.col('track').struct.field('album').alias('album')).with_columns(pl.col('album').struct.field('album_type').alias('album_type')).with_columns(pl.col('album').struct.field('artists').alias('artists')).with_columns(pl.col('artists').list.get(0).alias('first_artist_struct')).with_columns(pl.col('first_artist_struct').struct.field('name').alias('first_artist_name')).with_columns(pl.col('artists').list.get(1).alias('second_artist_struct')).with_columns(pl.col('second_artist_struct').struct.field('name').alias('second_artist_name')).with_columns(pl.col('artists').list.get(2).alias('third_artist_struct')).with_columns(pl.col('third_artist_struct').struct.field('name').alias('third_artist_name')).with_columns(pl.col('artists').list.get(3).alias('fourth_artist_struct')).with_columns(pl.col('fourth_artist_struct').struct.field('name').alias('fourth_artist_name')).with_columns(pl.col('track').struct.field('explicit').alias('is_explicit'))

# drop unnecessary columns
results = results.drop(columns=['is_local', 'primary_color', 'video_thumbnail', 'added_by', 'first_artist_struct', 'second_artist_struct', 'third_artist_struct', 'fourth_artist_struct', 'album', 'artists', 'track'])

# create a struct columns for all artists
results = results.with_columns(pl.struct(['first_artist_name', 'second_artist_name', 'third_artist_name', 'fourth_artist_name']).alias('artists'))

# drop all artist name columns
results = results.drop(columns=['first_artist_name', 'second_artist_name', 'third_artist_name', 'fourth_artist_name'])

# convert added_at column to datetime
results = results.with_columns(pl.col('added_at').str.to_datetime(strict=False))


results.head(5)

added_at,track_uri,track_name,album_type,is_explicit,artists
"datetime[μs, UTC]",str,str,str,bool,struct[4]
2023-11-03 04:00:00 UTC,"""spotify:track:6Db59Evpnl2VNJNgzVI3c3""","""The 312""","""single""",false,"{""Landis"",null,null,null}"
2023-11-03 04:00:00 UTC,"""spotify:track:6xKlgscNiEVJkDGd572KrI""","""Pump It Up (Mamacita)""","""album""",false,"{""BODYWORX"",""MOTi"",null,null}"
2023-11-03 04:00:00 UTC,"""spotify:track:14dRCRAvltxiNL7Mx0BDx8""","""Born Yesterday (Tritonia 452) [Tritonal Throwback]""","""album""",false,"{""Tritonal Tritonia Radio"",null,null,null}"
2023-11-03 04:00:00 UTC,"""spotify:track:7LUi7VNFBQpsJBfNY6iQ52""","""Past Life - Clean Bandit Remix""","""single""",false,"{""Felix Jaehn"",""Jonas Blue"",""Clean Bandit"",null}"
2023-11-03 04:00:00 UTC,"""spotify:track:02YBdthu3iJVFRBaXVxFO4""","""Lifetime""","""single""",false,"{""Marc Benjamin"",""David Allen"",null,null}"
